In [1]:
%load_ext autoreload
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
from dbmaster import MasterParams, MasterPersist
from dbbase import MusicDBIDModVal, MusicDBDir, MusicDBData
from dbnote import DownloadRecord, MergeSearchArtist, KnownRecord
from utils import FileIO, DirInfo, FileInfo, getFlatList, Timestat, TermTime, TermTimeTS, getTT
from pandas import Series, DataFrame, concat, Timestamp
from pandb import PanDBIO
from musicdb.rateyourmusic import MusicDBParams, RawWebData, MusicDBIO
from random import random, randint
from os import getpid
from functools import partial

mv = MusicDBIDModVal()
io = FileIO()
mpar = MusicDBParams()
dbio = MusicDBIO()
webio = RawWebData()
db = mpar.db

Setting Raw Path To [DirInfo(path=/Volumes/Seagate)]


/Users/tgadfort/code/dbmaster/src/dbmaster/paths.py:22: UserWarning: Perm Path [DirInfo(path=/Volumes/Piggy)] Does Not Exist.
  warnings.warn(f"Perm Path [{permDrivePath}] Does Not Exist.")


In [3]:
searchArtistRecord = DownloadRecord(db=db, name="SearchArtist", rTypes=["Index", "Data"])
downloadArtistRecord = DownloadRecord(db=db, name="DownloadArtist", rTypes=["Index"])
chartArtistRecord = KnownRecord(db=db, name="Charts")
knownArtists = dbio.rdio.getSummaryNameData
allArtists = partial(dbio.rdio.getData, "SearchArtist")

Setting Raw Path To [DirInfo(path=/Volumes/Seagate)]
DownloadRecord(db=RateYourMusic, name=SearchArtist, rTypes=['Index', 'Data'])
Setting Raw Path To [DirInfo(path=/Volumes/Seagate)]
DownloadRecord(db=RateYourMusic, name=DownloadArtist, rTypes=['Index'])
Setting Raw Path To [DirInfo(path=/Volumes/Seagate)]
DownloadRecord(db=RateYourMusic, name=Charts)


In [4]:
if False:
    from utils import FileInfo
    import shutil
    db = dbio.params.db
    dbdir = dbio.params.dir
    nameMap = {"MasterArtists": "SearchArtist", "LocalArtists": "DownloadArtist", "LocalAlbums": "DownloadAlbum"}
    nameMap = nameMap | {"LocalArtistTracks": "DownloadArtistTrack", "LocalArtistAlbums": "DownloadArtistAlbum",
                         "LocalArtistSimilar": "DownloadArtistSimilar", "LocalGenres": "DownloadGenre"}
    dinfo = DirInfo(f"/Users/tgadfort/anaconda3/envs/post/pandb/musicdb/{db}")
    
    cpMap = {}
    for origname, newname in nameMap.items():
        cpMap[f"{dbdir}SearchedFor{origname}"] = f"{db}-{newname}-Index"
        cpMap[f"{dbdir}SearchedFor{origname}Data"] = f"{db}-{newname}-Data"
        cpMap[f"{dbdir}SearchedFor{origname}Errors"] = f"{db}-{newname}-Error"
    assert dinfo.exists(), f"{dinfo} does not exist!"
    for srcFile, dstFile in cpMap.items():
        srcFile = dinfo.join(f"{srcFile}.p")
        dstFile = dinfo.join(f"{dstFile}.p")
        if srcFile.exists():
            srcFile.mvFile(dstFile)

In [ ]:
##########################################################################################
# Show Summary
##########################################################################################
print(f"{db} Search Results (PID={getpid()})".format(db))
searchArtistRecord.info()
downloadArtistRecord.info()
chartArtistRecord.info()
print(f"  {'KnownArtist Data': <20}: {knownArtists().shape[0]}")

# Search For New Artists

In [4]:
######################################################################################################
# Explode Recent Data
######################################################################################################
def getNewData(searchArtistRecord):
    newData = searchArtistRecord.getData()
    if len(newData) == 0:
        return DataFrame()
    newData = concat([DataFrame(s).T for s in newData.values()])
    newData.index = newData.index.astype(str)
    newData = newData[~newData.index.duplicated()]
    return newData

def mergeNewData(searchArtistRecord):
    searchArtistRecord.load(verbose=False)
    newData = getNewData(searchArtistRecord)
    msr = MergeSearchArtist(db)
    msr.mergeLocal(searchArtistRecord, newData, test=False)

In [45]:
dbio = MusicDBIO(verbose=False,local=True,mkDirs=False)
webio = RawWebData(debug=False)
knownNames = PanDBIO().getUniqueUnmatchedArtistNames(db)
mergeNewData(searchArtistRecord)
searchedNames = Series(searchArtistRecord.getIndex())
artistNamesToGet = knownNames[~knownNames.map(searchArtistRecord.isKnown)]

print(f"# {db} Search Results (PID={getpid()})")
print(f"#   Available Names:     {knownNames.shape[0]}")
print(f"#   Known Artist Names:  {searchedNames.shape[0]}")
print(f"#   Artist Names To Get: {artistNamesToGet.shape[0]}")

del searchedNames
del knownNames

/Users/tgadfort/code/dbmaster/src/dbmaster/paths.py:22: UserWarning: Perm Path [DirInfo(path=/Volumes/Piggy)] Does Not Exist.
  warnings.warn(f"Perm Path [{permDrivePath}] Does Not Exist.")


Setting Raw Path To [DirInfo(path=/Volumes/Seagate)]
Setting Raw Path To [DirInfo(path=/Volumes/Seagate)]
Setting Raw Path To [DirInfo(path=/Volumes/Seagate)]
*****************************************************************************************************************************
*                                            MergeSearchArtist(db=RateYourMusic)                                            *
*****************************************************************************************************************************
  ==> Creating copy of local SearchArtist data
Setting Raw Path To [DirInfo(path=/Volumes/Seagate)]


/Users/tgadfort/code/dbmaster/src/dbmaster/paths.py:22: UserWarning: Perm Path [DirInfo(path=/Volumes/Piggy)] Does Not Exist.
  warnings.warn(f"Perm Path [{permDrivePath}] Does Not Exist.")


  ==> Created copy of local SearchArtist data with (642090, 2) shape
Setting Raw Path To [DirInfo(path=/Volumes/Seagate)]


/Users/tgadfort/code/dbmaster/src/dbmaster/paths.py:22: UserWarning: Perm Path [DirInfo(path=/Volumes/Piggy)] Does Not Exist.
  warnings.warn(f"Perm Path [{permDrivePath}] Does Not Exist.")


Found 479 Searches
Found 9524 Results
Found 0 Duplicate Results
Found 642090 Previously Known Results
Found 1720 New Results
  ==> 19.88 All Results / Search
  ==> 3.59 New Results / Search
Saving 643810 Data ... Setting Raw Path To [DirInfo(path=/Volumes/Seagate)]


/Users/tgadfort/code/dbmaster/src/dbmaster/paths.py:22: UserWarning: Perm Path [DirInfo(path=/Volumes/Piggy)] Does Not Exist.
  warnings.warn(f"Perm Path [{permDrivePath}] Does Not Exist.")


Done
Setting Raw Path To [DirInfo(path=/Volumes/Seagate)]
*****************************************************************************************************************************
*                                                 Initializing SearchArtist                                                 *
*****************************************************************************************************************************
  Initializing Data   ... Done
# RateYourMusic Search Results (PID=84599)
#   Available Names:     1081280
#   Known Artist Names:  56105
#   Artist Names To Get: 1025157


In [44]:
ts = Timestat(f"Getting {db} ArtistIDs")
tt = getTT(skipEOD=False, vacation=True)
assert dbio.rdio.isLocal, f"MusicDBIO is not set for local downloads!"

def sleeptime():
    return 32+10*random()
    
def isError(artistName, nErrors, sleeptime, error=None):
    searchArtistRecord.setError(index=artistName)
    print(f"Search Error ==> {artistName}: {error}")
    nErrors.append(artistName)
    webio.sleep(sleeptime)
    
n = 0
maxN = 25000000
nErrors = []
searchArtistRecord.load(verbose=False)
for i, (idx, artistName) in enumerate(artistNamesToGet.items()):
    if searchArtistRecord.isKnown(artistName):
        continue

    if len(nErrors) >= 3:
        print("Stopping due to 3 consecutive errors")
        break

    print(f"{i: <8} | {n: <8} | ", end="")
    
    try:
        response = webio.getArtistSearchData(artistName=artistName)
    except Exception as error:
        stime = len(nErrors)*randint(30,60)+sleeptime()
        isError(artistName, nErrors, stime, error)
        continue

    if not isinstance(response, dict):
        stime = len(nErrors)*randint(30,60)+sleeptime()
        isError(artistName, nErrors, stime, "NotDict")
        continue

    nErrors = []
    searchArtistRecord.setData(index=artistName, data=response)
    n += 1
    if n >= maxN:
        print(f"Breaking after {maxN} downloads...")
        break
    webio.sleep(sleeptime())
    searchArtistRecord.save(verbose=False)
    if tt.isFinished():
        break    

        
    if (i+1) % randint(6,8) == 0:
        webio.sleep(sleeptime())
    
    if (i+1) % randint(28,37) == 0:
        webio.sleep(sleeptime())
        
    if n % 15 == 0:
        ts.update(n=n)
        stime = 150 if (n+1) % 125 == 0 else 150
        webio.sleep(stime)
    
ts.stop()
searchArtistRecord.save()

 [Getting RateYourMusic ArtistIDs] | StartTime 2023-12-04 11:37:03
   ====> Terminate Time Set To 2023-12-04 19:00:00 <====
   ====> Will Terminate Process 7 Hour and 22 Minute From Now
0        | 0        | Searching For Adam Daniel                                       25
1        | 1        | Searching For F.L.Y.                                            25
2        | 2        | Searching For Jerome Noak                                       25
3        | 3        | Searching For Ring, Cicada                                      25
4        | 4        | Searching For Cannonball Jane                                   25
5        | 5        | Searching For John Rivera                                       25
6        | 6        | Searching For Steffen Kahles                                    25
7        | 7        | Searching For Bert Shefter                                      25
8        | 8        | Searching For Leniwiec                                          22
9        | 9 

In [14]:
searchArtistRecord.save()

*****************************************************************************************************************************
*                                                    Saving SearchArtist                                                    *
*****************************************************************************************************************************
  Saving Index [45457] ... Done
  Saving Data [750] ... Done
  Saving Error [18] ... Done


In [12]:
msa = MergeSearchArtist(dbio.db)
msa.copyFromLocal(force=True)

*****************************************************************************************************************************
*                                            MergeSearchArtist(db=RateYourMusic)                                            *
*****************************************************************************************************************************
New Local Shape:  (584812, 2)
Old Global Shape: (440675, 2)
New Global Shape: (584812, 2)


# Download Artist Data

In [14]:
dbio = MusicDBIO(verbose=False,local=True,mkDirs=False)
webio = RawWebData(debug=False)
allArtists = partial(dbio.rdio.getData, "SearchArtist")
knownNames = allArtists()
knownNames["URL"] = knownNames["Ref"].map(lambda x: f"https://rateyourmusic.com{x}" if isinstance(x, str) else None)
knownNames['ModVal'] = knownNames.index.map(mv.getModVal)
availableNames = knownNames[~knownNames["URL"].map(downloadArtistRecord.isKnown)][["Name", "URL", "ModVal"]]
artistNamesToGet = Series({modVal: modValDF for modVal,modValDF in availableNames.groupby("ModVal")})

print(f"# {db} Search Results")
print(f"#   Available Names:     {knownNames.shape[0]}")
print(f"#   Known Artist Names:  {downloadArtistRecord.numKnown()}")
print(f"#   Artist Names To Get: {availableNames.shape[0]}")

#del availableNames
#del downloadedNames
#del knownNames

/Users/tgadfort/code/dbmaster/src/dbmaster/paths.py:22: UserWarning: Perm Path [DirInfo(path=/Volumes/Piggy)] Does Not Exist.
  warnings.warn(f"Perm Path [{permDrivePath}] Does Not Exist.")


Setting Raw Path To [DirInfo(path=/Volumes/Seagate)]
# RateYourMusic Search Results
#   Available Names:     643810
#   Known Artist Names:  307752
#   Artist Names To Get: 342295


In [ ]:
ts = Timestat(f"Getting {db} ArtistIDs")
tt = getTT(skipEOD=False, vacation=True)
assert dbio.rdio.isLocal, f"MusicDBIO is not set for local downloads!"

def sleeptime():
    return 32+10*random()
    
def isError(artistURL, artistName, nErrors, sleeptime, error=None):
    downloadArtistRecord.setError(index=artistURL)
    print(f"Download Error ==> {artistName}: {error}")
    nErrors.append(artistURL)
    webio.sleep(sleeptime)
    
n = 0
maxN = 2500000
stop = False
nErrors = []
for modVal,modValData in artistNamesToGet.items():
    #modVal = groupModVal[0]
    if stop is True:
        break
    N = modValData.shape[0]
    for i,(artistID,row) in enumerate(modValData.iterrows()):
        artistURL = row["URL"]
        artistName = row["Name"]
        
        if downloadArtistRecord.isKnown(artistURL):
            continue
    
        if len(nErrors) >= 5:
            print("Stopping due to 5 consecutive errors")
            stop = True
            break

        print(f"{modVal: <3} | {i: <5} | {N: <5} | {n: <5} | ", end="")        
        
        try:
            response = webio.getArtistData(artistID=artistID, artistName=artistName, artistURL=artistURL)
        except Exception as error:
            isError(artistURL, artistName, nErrors, 2*sleeptime(), error)
            continue
    
        if not isinstance(response, bytes):
            isError(artistURL, artistName, nErrors, 2*sleeptime(), "NotBytes")
            continue

        nErrors = []
        downloadArtistRecord.setIndex(index=artistURL)
        dbio.rdio.saveData("RawArtist", modVal, artistID, data=response)
        
        n += 1
        if n >= maxN:
            print(f"Breaking after {maxN} downloads...")
            stop = True
            break
            
        downloadArtistRecord.save(verbose=False)
        webio.sleep(sleeptime())
        if tt.isFinished():
            stop = True
            break    
    
            
        if (i+1) % randint(6,8) == 0:
            webio.sleep(sleeptime())
        
        if (i+1) % randint(28,37) == 0:
            webio.sleep(sleeptime())
            
        if n % 15 == 0:
            ts.update(n=n)
            stime = 150 if (n+1) % 125 == 0 else 150
            webio.sleep(stime)

ts.stop()
downloadArtistRecord.save()

 [Getting RateYourMusic ArtistIDs] | StartTime 2023-12-05 07:04:01
   ====> Terminate Time Set To 2023-12-05 19:00:00 <====
   ====> Will Terminate Process 11 Hour and 55 Minute From Now
0   | 0     | 3137  | 0     | Getting Artist Data For Jay Daversa (1184100)                             135137
0   | 1     | 3137  | 1     | Getting Artist Data For Eric Watson (161800)                              183932
0   | 2     | 3137  | 2     | Getting Artist Data For Crown of Thorns (65300)                           154806
0   | 3     | 3137  | 3     | Getting Artist Data For Paul Rader (1363700)                              135173
0   | 4     | 3137  | 4     | Getting Artist Data For Lichte Dichter (1568000)                          146099
0   | 5     | 3137  | 5     | Getting Artist Data For Adrienne Torf (216700)                            146895
0   | 6     | 3137  | 6     | Getting Artist Data For Michael Morgan (208900)                           167381
0   | 7     | 3137  | 7     | Gettin

In [13]:

downloadArtistRecord.save()

*****************************************************************************************************************************
*                                                   Saving DownloadArtist                                                   *
*****************************************************************************************************************************
  Saving Index [307751] ... Done
  Saving Error [1] ... Done


In [ ]:
ts = Timestat(f"Getting {db} ArtistIDs")
tt = getTT(skipEOD=False, vacation=True)
assert dbio.rdio.isLocal, f"MusicDBIO is not set for local downloads!"

def sleeptime():
    return 32+10*random()
    
def isError(artistName, nErrors, sleeptime, error=None):
    searchArtistRecord.setError(index=artistName)
    print(f"Search Error ==> {artistName}: {error}")
    nErrors.append(artistName)
    webio.sleep(sleeptime)
    
n = 0
maxN = 25000000
nErrors = []
searchArtistRecord.load(verbose=False)
for i, (idx, artistName) in enumerate(artistNamesToGet.items()):
    if searchArtistRecord.isKnown(artistName):
        continue

    if len(nErrors) >= 3:
        print("Stopping due to 3 consecutive errors")
        break

    print(f"{i: <8} | {n: <8} | ", end="")
    
    try:
        response = webio.getArtistSearchData(artistName=artistName)
    except Exception as error:
        stime = len(nErrors)*randint(30,60)+sleeptime()
        isError(artistName, nErrors, stime, error)
        continue

    if not isinstance(response, dict):
        stime = len(nErrors)*randint(30,60)+sleeptime()
        isError(artistName, nErrors, stime, "NotDict")
        continue

    nErrors = []
    searchArtistRecord.setData(index=artistName, data=response)
    n += 1
    if n >= maxN:
        print(f"Breaking after {maxN} downloads...")
        break
    webio.sleep(sleeptime())
    searchArtistRecord.save(verbose=False)
    if tt.isFinished():
        break    

        
    if (i+1) % randint(6,8) == 0:
        webio.sleep(sleeptime())
    
    if (i+1) % randint(28,37) == 0:
        webio.sleep(sleeptime())
        
    if n % 15 == 0:
        ts.update(n=n)
        stime = 150 if (n+1) % 125 == 0 else 150
        webio.sleep(stime)
    
ts.stop()
searchArtistRecord.save()

In [ ]:
downloadedNames

In [ ]:
from random import random, randint
from fileutils import DirInfo, FileInfo
from ioutils import FileIO
from timeutils import Timestat
from apiutils import WebIO

def sleeptime():
    return 45+10*random()

io       = FileIO()
saveDir  = DirInfo("/Users/tgadfort/Desktop/RateYourMusic/Auto")
N        = artistNamesToGet.shape[0]
webio      = WebIO()
n        = 0
nErrors  = []
nLowSize = []

ts = Timestat(f"Downloading {N} RateYourMusic Files")

localArtistsDict = localArtists.get()
errorsDict       = errors.get()

for i,(ref,name) in enumerate(artistNamesToGet.iteritems()):
    if len(nLowSize) >= 3:
        print(f"Stopping after {len(nLowSize)} low sizes (len(bytes) < 120k).")
        break
    if len(nErrors) >= 3:
        print(f"Stopping after {len(nErrors)} errors (Code != 200).")
        break
        
    url  = f"https://rateyourmusic.com{ref}" if ref.startswith("/artist") else ref
    if not url.startswith("http"):
        continue
    url  = fixURL(url)
#    if any([dct.get(url) is not None for dct in [localArtistsDict]]):
#        continue
    if any([dct.get(url) is not None for dct in [localArtistsDict,errorsDict]]):
        continue
        
    name = name.replace("/","_")
    savename = saveDir.join(f"{name}.p")
    if savename.exists():
        localArtistsDict[url] = True
        continue
        
    print(f"{i: <8} | {n: <8} | {N: <8} | Downloading {url: <75}  ===>  ", end="")
    retval = webio.get(url)
    if retval.code != 200:
        print(f"Error code {retval.code}")
        nErrors.append(retval.code)
        errorsDict[url] = True
        webio.sleep(len(nErrors)*randint(30,60)+sleeptime())
        continue
        
    numBytes = len(retval.data)
    if numBytes < 100000:
        nLowSize.append(numBytes)
    else:
        nLowSize = []

    n += 1
    nErrors = []
    nLowSize = []
    localArtistsDict[url] = True
    io.save(idata=retval.data, ifile=savename)
    print(f"✓ ({len(retval.data)})")
    webio.sleep(sleeptime())
    
    if i % randint(4,8) == 0:
        webio.sleep(sleeptime())
    
    if i % randint(15,37) == 0:
        webio.sleep(sleeptime())
        

    if (n+1) % 15 == 0:
        saveArtistData(db, localArtistsDict, errorsDict)        
        stime = 500 if (n+1) % 45 == 0 else 150
        webio.sleep(stime)
        try:
            ts.update(n=i, N=N)
        except:
            pass
        
if True: saveArtistData(db, localArtistsDict, errorsDict)
ts.stop()

In [ ]:
ts = Timestat(f"Getting {db} ArtistIDs")
tt = getTT(skipEOD=False, vacation=True)
assert dbio.rdio.isLocal, f"MusicDBIO is not set for local downloads!"

def isError(artistName, nErrors, sleeptime, error=None):
    searchArtistRecord.setError(index=artistName)
    print(f"Search Error ==> {artistName}: {error}")
    nErrors.append(artistName)
    webio.sleep(sleeptime)
    
n = 0
maxN = 2500000
stop = False
nErrors = []
#downloadArtistRecord.load()
for groupModVal,modValData in artistNamesToGet.items():
    if stop is True:
        break
    N = modValData.shape[0]
    for i,(artistID,artistName) in enumerate(modValData['name'].items()):    
        if downloadArtistRecord.isKnown(artistName):
            continue
    
        if len(nErrors) >= 5:
            print("Stopping due to 5 consecutive errors")
            break
    
        print(f"{groupModVal: <8} | {i: <8} | {N: <8} | {n: <8} | ", end="")
        response = webio.getArtistData(artistID=artistID, artistName=artistName)        
        stop = True
        break
        
        try:
            response = webio.getArtistData(artistID=artistID, artistName=artistName)        
        except Exception as error:
            isError(artistName, nErrors, 5, error)
            continue
    
        if not isinstance(response, dict):
            isError(artistName, nErrors, 3.5, "NotDict")
            continue

        
        nErrors = []
        dbio.rdio.saveData("RawArtist", data=response, modval=modVal, dbID=artistID)
        downloadArtistRecord.setIndex(index=artistName)
        webio.sleep(2.0)
        n += 1
            
        if n % 25 == 0:
            ts.update(n=n)
            searchArtistRecord.save()
            webio.wait(15.0)
            if tt.isFinished():
                break
        
        if n >= maxN:
            print("Breaking after {0} downloads...".format(maxN))
            break

ts.stop()
#downloadArtistRecord.save()

In [ ]:
dbio.rdio.saveData("RawArtist", data=response, modval=99, dbID='99')

In [ ]:
ts = Timestat("Getting {0} ArtistIDs".format(db))
tt = getTT(skipEOD=False)

n    = 0
maxN = 2500000
localArtistsDict = localArtists.get()
localArtistsDictErrors = localArtistsErrors.get()
stop = False
nErrors = []
for groupModVal,modValData in artistNamesToGet.iteritems():
    if stop is True:
        break
    for j,(artistID,row) in enumerate(modValData.iterrows()):
        if len(nErrors) >= 5:
            for artistID in nErrors:
                print(f"del searchedForErrors['{artistID}']")
                stop=True
                break
        if any([dct.get(artistID) is not None for dct in [localArtistsDict, localArtistsDictErrors]]):
            continue
            
        artistName  = row["Name"]
        artistRef   = row["Ref"]

        print(f"{groupModVal: <8} |{j: <8} | {n: <8} | ", end="")
        try:
            response = webio.getArtistData(artistName=artistName, artistRef=artistRef)
        except:
            print("Error ==> {0}".format(artistName))
            localArtistsDictErrors[artistID] = True
            nErrors.append(artistID)
            webio.sleep(10)
            continue

        if not isinstance(response,bytes):
            print("Error ==> {0}".format(artistName))
            localArtistsDictErrors[artistID] = True
            nErrors.append(artistID)
            webio.sleep(3.5)
            continue

        nErrors = []
        modVal  = mv.getModVal(artistID)
        dbio.data.saveRawArtistData(data=response, modval=modVal, dbID=artistID)
        localArtistsDict[artistID] = True
        webio.sleep(4.5)
        n += 1
        nLastErrors = 0
        
        if n % 5 == 0:
            if tt.isFinished():
                stop=True
                break

        if n % 50 == 0:
            webio.sleep(5)
            
        if n % 100 == 0:
            saveArtistData(db, localArtistsDict, localArtistsDictErrors)
            if tt.isFinished():
                stop=True
                break
            webio.wait(10.0)

        if n >= maxN:
            print("Breaking after {0} downloads...".format(maxN))
            stop=True
            break

ts.stop()
if True: saveArtistData(db, localArtistsDict, localArtistsDictErrors)

In [ ]:
if True: saveArtistData(db, localArtistsDict, localArtistsDictErrors)

# Download Album Data

## Create Media Data

In [ ]:
mediaData = {}
for modVal in range(100):
    modValData = dbio.data.getModValData(modVal)
    modValMediaData = {}
    for artistID,artistIDData in modValData.iteritems():
        for mediaType,mediaTypeData in artistIDData.media.media.items():
            modValMediaData.update({code: [artistID,media.album,media.url] for code,media in mediaTypeData.items()})
    mediaData.update(modValMediaData)
    if (modVal+1) % 10 == 0:
        print(f"ModVal = {modVal+1}")
        
df = DataFrame(mediaData).T
df.columns = ["ArtistID", "Name", "Ref"]
knownMedia.save(data=df)

## Download Data

In [ ]:
dbio   = bandcamp.MusicDBIO(verbose=False,local=True,mkDirs=False)
webio = bandcamp.RawWebData(debug=False)

In [ ]:
useArtist = False
numMaster = 250

knownAlbumsData = knownMedia.get()
knownAlbumsData['IndexModVal'] = knownAlbumsData.index.map(dbio.getModVal)
availableNames  = concat([artistIDDF.head(numMaster) for artistID,artistIDDF in knownAlbumsData.groupby(["ArtistID"])])
localAlbumsDict = localAlbums.get()
availableNames  = availableNames[~availableNames.index.isin(localAlbumsDict.keys())]
albumNamesToGet = Series({modVal: modValDF for modVal,modValDF in availableNames.groupby(["IndexModVal"])})

print(f"# {db} Album Search Results")
print(f"#   Available Album IDs:  {knownAlbumsData.shape[0]}")
print(f"#   Known Album IDs:      {len(localAlbumsDict)}")
print(f"#   Albums To Download:   {availableNames.shape[0]}")

del availableNames
del localAlbumsDict
del knownAlbumsData

#   Albums To Download:   23457
#   Albums To Download:   20437
#   Albums To Download:   12137
#   Albums To Download:   4132

In [ ]:
def saveAlbumData(db, localAlbumsDict, searchedForErrors):
    print("="*150)
    print(f"Saving {len(localAlbumsDict)} {db} Albums Data")
    localAlbums.save(data=localAlbumsDict)
    print(f"Saving {len(searchedForErrors)} {db} Searched For Errors")
    errors.save(data=searchedForErrors)
    print("="*150)

In [ ]:
ts = Timestat("Getting {0} AlbumIDs".format(db))
tt = getTT(skipEOD=False)

n    = 0
maxN = 25000000
localAlbumsDict     = localAlbums.get()
searchedForErrors   = errors.get()
stop = False
nErrors = []
for groupModVal,modValData in albumNamesToGet.iteritems():
    if stop is True:
        break
    for j,(albumID,row) in enumerate(modValData.iterrows()):
        if len(nErrors) >= 5:
            for artistID in nErrors:
                print(f"del searchedForErrors['{artistID}']")
                stop=True
                break
        if any([dct.get(albumID) is not None for dct in [localAlbumsDict, searchedForErrors]]):
            continue
            
        artistID   = row["ArtistID"]
        albumName  = row["Name"]
        albumRef  = row["Ref"]

        print(f"{groupModVal: <8} |{j: <8} | {n: <8} | ", end="")
        try:
            response = webio.getAlbumData(albumName=albumName, albumRef=albumRef)
        except:
            print("Error ==> {0}".format(albumName))
            searchedForErrors[albumID] = True
            nErrors.append(albumID)
            webio.sleep(10)
            continue

        if not isinstance(response,bytes):
            print("Error ==> {0}".format(albumName))
            searchedForErrors[albumID] = True
            nErrors.append(albumID)
            webio.sleep(3.5)
            continue

        nErrors = []
        modVal=dbio.mv.get(albumID)
        dbio.data.saveRawArtistAlbumData(data=response, modval=modVal, dbID=albumID)
        localAlbumsDict[albumID] = True
        webio.sleep(4.5)
        n += 1
        nLastErrors = 0
        
        if n % 5 == 0:
            if tt.isFinished():
                stop=True
                break

        if n % 50 == 0:
            webio.sleep(5)
            
        if n % 100 == 0:
            saveAlbumData(db, localAlbumsDict, searchedForErrors)
            if tt.isFinished():
                stop=True
                break
            webio.wait(10.0)

        if n >= maxN:
            print("Breaking after {0} downloads...".format(maxN))
            stop=True
            break

ts.stop()
if True: saveAlbumData(db, localAlbumsDict, searchedForErrors)

In [ ]:
from os import getpid
getpid()

# Download Lists

## Download Starter

In [ ]:
from apiutils import webio
from ioutils import FileIO, HTMLIO
io  = FileIO()
hio = HTMLIO()
webio = webio()

In [ ]:
starter             = {}
starter["List"]     = "https://www.bandcamp.org/lists.php"
starter["Genre"]    = "https://www.bandcamp.org/genre.php"
#starterBestAlbum = "https://www.bandcamp.org/ratings/6-highest-rated/2023/1"
starter["Rating"]   = "https://www.bandcamp.org/ratings"
starter["Discover"] = "https://www.bandcamp.org/discover"
starter["Releases"] = "https://www.bandcamp.org/releases"
starter["MustHear"] = "https://www.bandcamp.org/must-hear"

savename = "../../sandbox/AOTYstarter.p"
starterData = io.get(savename)
for key,url in starter.items():
    if starterData.get(key) is not None:
        continue
    retval = webio.get(url)
    if retval.code == 200:
        print(key)
        starterData[key] = retval.data
    webio.sleep(3)
        
print(f"Saving data to {savename}")
io.save(idata=starterData, ifile=savename)

In [ ]:
useStarter = False
useSite    = True

if useStarter is True:
    aotyData = io.get("../../sandbox/AOTYstarter.p")
elif useSite is True:
    aotyData = io.get("../../sandbox/AOTYsiteData.p") | io.get("../../sandbox/AOTYsiteData2.p")
else:
    aotyData = {}
    
N  = len(aotyData)
ts = Timestat(f"Sorting {N} Site Refs")
refsData = {"List": {}, "Lists": {}, "Rating": {}, "Album": {}, "Artist": {}, "Discover": {}, "Genre": {}, "MustHear": {}, "Release": {}, "Spotify": {}, "Apple": {}, "Amazon": {}, "User": {}}
for n,(key,keyData) in enumerate(aotyData.items()):
    if (n+1) % 500 == 0 or (n+1) == 100:
        ts.update(n=n+1, N=N)
        
    bsdata = hio.get(keyData)
    refs = bsdata.findAll("a")
    for ref in refs:
        href = ref.get('href')
        if not isinstance(href,str):
            continue
        if href.startswith("/list/"):
            refsData["List"][href] = ref.text.strip()
        elif "lists.php" in href:
            refsData["Lists"][href] = ref.text.strip()
        elif href.startswith("/ratings/"):
            refsData["Rating"][href] = ref.text.strip()
        elif href.startswith("/artist/"):
            refsData["Artist"][href] = ref.text.strip()
        elif href.startswith("/album/"):
            refsData["Album"][href] = ref.text.strip()
        elif href.startswith("/discover/"):
            refsData["Discover"][href] = ref.text.strip()
        elif href.startswith("/genre/"):
            refsData["Genre"][href] = ref.text.strip()
        elif href.startswith("/must-hear/"):
            refsData["MustHear"][href] = ref.text.strip()
        elif "/releases/" in href:
            refsData["Release"][href] = ref.text.strip()
        elif "spotify.com" in href:
            refsData["Spotify"][href] = ref.text.strip()
        elif "apple.com" in href:
            refsData["Apple"][href] = ref.text.strip()
        elif "amazon.com" in href:
            refsData["Amazon"][href] = ref.text.strip()
        elif "/user/" in href:
            refsData["User"][href] = ref.text.strip()
        else:
            continue
            print(href,'\t|\t',ref.text)
            
ts.stop()

In [ ]:
savename = "../../sandbox/AOTYsiteRefs.p"
print(f"Saving data to {savename}")
for key,keyData in refsData.items():
    print(f"  {key: <20}{len(keyData)}")
io.save(idata=refsData, ifile=savename)

In [ ]:
siteData   = {}
artistData = {}
albumData  = {}
userData   = {}
otherData  = {}

if False:
    for ref,name in refsData["List"].items():
        if ref.startswith("/artist/"):
            artistData[ref] = name
        elif ref.startswith("/album/"):
            albumData[ref] = name
        elif ref.startswith("/user/"):
            userData[ref] = name
        elif ref.startswith("/"):
            siteData[ref] = name
        else:
            otherData[ref] = name
else:
    for key,keyData in refsData.items():
        for ref,name in keyData.items():
            if ref.startswith("/artist/"):
                artistData[ref] = name
            elif ref.startswith("/album/"):
                albumData[ref] = name
            elif ref.startswith("/user/"):
                userData[ref] = name
            elif ref.startswith("/"):
                if "/list/" in ref:
                    siteData[ref] = name
                else:
                    otherData[ref] = name
            else:
                otherData[ref] = name
            
print(f"Found {len(artistData)} Artist Refs")
print(f"Found {len(albumData)} Album Refs")
print(f"Found {len(userData)} User Refs")
print(f"Found {len(siteData)} Site Refs")
print(f"Found {len(otherData)} Other Refs")

In [ ]:
for year in range(1970,2005):
    key = f'/lists.php?y={year}'
    val = 'View More'
    siteData[key] = val

In [ ]:
savename1 = "../../sandbox/AOTYsiteData.p"
siteDataDownloads1 = io.get(savename1)
print(f"Found {len(siteDataDownloads1)} Previous Downloads")
savename2 = "../../sandbox/AOTYsiteData2.p"
siteDataDownloads2 = io.get(savename2)
print(f"Found {len(siteDataDownloads2)} Previous Downloads")
N = len(siteData)
ts = Timestat(f"Downloading {N} Site Refs")
for n,(ref,name) in enumerate(siteData.items()):
    url=f"https://www.bandcamp.org{ref}"
    if any([dct.get(ref) is not None for dct in [siteDataDownloads1,siteDataDownloads2]]):
        continue
    
    retval = webio.get(url)
    if retval.code == 200:
        print(f"{n: <6} | {N: <6} | {ref}")
        siteDataDownloads2[ref] = retval.data
    webio.sleep(3)
    
    if (n+1) % 25 == 0:
        ts.update(n=n+1,N=N)
        print(f"Saving {len(siteDataDownloads2)} data to {savename2}")
        io.save(idata=siteDataDownloads2, ifile=savename2)

ts.stop()
        
print(f"Saving data to {savename2}")
io.save(idata=siteDataDownloads2, ifile=savename2)

In [ ]:
siteDataDownloads = io.get("../../sandbox/AOTYsiteData.p")

In [ ]:
io.save(idata=siteDataDownloads, ifile=savename)

## Check For New Data

In [ ]:
from lib.bandcamp import MusicDBID
mid = MusicDBID()
mid.getAlbumID('/album/515536-beyonce-renaissance/critic-lists/?f=all&y=2022')

In [ ]:
df = DataFrame(Series(refsData["Album"])).reset_index().rename(columns={"index": "Ref", 0: "List"})
df["AlbumID"] = df["Ref"].map(mid.getAlbumID)
df = df[~df["AlbumID"].duplicated()]

In [ ]:
df = DataFrame(Series(refsData["Artist"])).reset_index().rename(columns={"index": "Ref", 0: "Name"})
df["ArtistID"] = df["Ref"].map(mid.getArtistID)
df = df[~df["ArtistID"].duplicated()]

In [ ]:
artistNames = searchArtists()

In [ ]:
df.index = df["ArtistID"]
df = df.drop(["ArtistID"], axis=1)
artistNames = concat([artistNames,df])
artistNames = artistNames[~artistNames.index.duplicated()]

In [ ]:
dbio.data.saveSearchArtistData(data=artistNames)

# Backup

In [ ]:
from utils import StoreData, backup
from numpy import array_split
sd = StoreData("bandcamp", "Artist")
for modVals in array_split(range(100), 2):
    sd.mergeLocalData(modVals=modVals)
sd.mergeGlobalData()

In [ ]:
from time import sleep
sleep(200)
sd.mergeGlobalData()

In [ ]:
for modVal in range(67):
    srcDir = DirInfo(f"/Volumes/Piggy/Discog/artists-bandcamp/{modVal}/artists")
    files  = [FileInfo(ifile) for ifile in srcDir.getFiles()]
    files  = [finfo for finfo in files if finfo.basename.isdigit()]
    dstDir = DirInfo(f"/Users/tgadfort/Music/Discog/artists-bandcamp/{modVal}/artists")
    for srcFile in files:
        dstFile = dstDir.join(srcFile.name)
        srcFile.mvFile(dstFile)